In [14]:
import os, sys, pysam, matplotlib.pyplot as plt, numpy as np, pandas as pd, seaborn as sns
from glob import glob
from Bio import SeqIO

In [15]:
def get_motifs(ref, seq, motif2pos):
    """Return position of motifs in sequence"""
    motifs = {m: set() for m in motif2pos}
    motifs_lens = set(len(m) for m in motifs)#; print(motifs_lens)
    # only count + strand
    strand = "+"
    for s in range(len(seq)-min(motifs_lens)):
        for l in motifs_lens:
            m = seq[s:s+l]
            if m in motifs:
                pos = "%s:%s%s"%(ref, s+motif2pos[m], strand)
                motifs[m].add(pos)
    return motifs

def fasta2data(fasta, motif2pos):
    """Return """
    s2d, s2c = {}, {}
    for r in SeqIO.parse(fasta, "fasta"):
        ref = r.name
        seq = str(r.seq)
        family, species = r.description.split()[1:3]
        spname = "%s%s"%(family.lower()[0], species)
        sys.stderr.write(" %s %s     \r"%(spname, ref))
        # get motifs
        motifs = get_motifs(ref, seq, motif2pos)
        # update species motifs
        if spname in s2d: 
            for m in motif2pos: 
                s2d[spname][m] = s2d[spname][m].union(motifs[m])
        # all motifs are fresh here
        else: 
            s2d[spname] = motifs #{m: set() for m in motif2pos}
            s2c[spname] = []
        s2c[spname].append(ref)
    return s2d, s2c

fasta = "/home/lpryszcz/cluster/dna_mods/ref/mock_community.fa"

mod2mers = {"6mA": ((2, "GATC"), ), "5mC": ((2, "CCAGG"), (2, "CCTGG"), (1, "CG"))}; mod2mers
motif2pos = {m: p for mod in mod2mers for p, m in mod2mers[mod]}; motif2pos

species2motifs, species2refs = fasta2data(fasta, motif2pos)

In [16]:
# #/home/lpryszcz/cluster/dna_mods/ecoli/modPhred.2020/PRJNA477598/
fn = "/home/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJEB22772/mod.gz" # PRJNA477598 PRJEB22772
df = pd.read_csv(fn, sep="\t", skiprows=16)#; df.head() 
# get ref as chr:pos + strand
df["ref"] = df['chr'].astype(str) + ":" + df['pos'].astype(str) + df['strand']; df.head()

,chr,pos,ref_base,strand,mod,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJEB22772/minimap2/MARC_ZFscreens_R9.4_1D-Ecoli-run_FAF05145.bam depth,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJEB22772/minimap2/MARC_ZFscreens_R9.4_1D-Ecoli-run_FAF05145.bam basecall_accuracy,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJEB22772/minimap2/MARC_ZFscreens_R9.4_1D-Ecoli-run_FAF05145.bam mod_frequency,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJEB22772/minimap2/MARC_ZFscreens_R9.4_1D-Ecoli-run_FAF05145.bam median_mod_prob,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJEB22772/minimap2/MARC_ZFscreens_R9.4_2D-Ecoli-run_FAF05711.bam depth,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJEB22772/minimap2/MARC_ZFscreens_R9.4_2D-Ecoli-run_FAF05711.bam basecall_accuracy,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJEB22772/minimap2/MARC_ZFscreens_R9.4_2D-Ecoli-run_FAF05711.bam mod_frequency,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJEB22772/minimap2/MARC_ZFscreens_R9.4_2D-Ecoli-run_FAF05711.bam median_mod_prob,ref
0,NC_000913.3,420,C,+,5mC,467,0.983,0.719,0.967,132,0.962,0.697,0.967,NC_000913.3:420+
1,NC_000913.3,422,C,-,5mC,361,0.623,0.352,0.833,103,0.670,0.427,0.900,NC_000913.3:422-
2,NC_000913.3,423,C,-,5mC,461,0.996,0.028,0.600,121,0.983,0.058,0.667,NC_000913.3:423-
3,NC_000913.3,475,C,+,5mC,438,0.945,0.664,0.933,131,0.947,0.687,0.967,NC_000913.3:475+
4,NC_000913.3,477,C,-,5mC,446,0.922,0.538,0.900,119,0.983,0.420,0.867,NC_000913.3:477-


In [17]:
min_freq = 0.05
spname = "ecoli"
for spcol in filter(lambda c: c.endswith("mod_frequency"), df.columns):
    #spname = spcol.split("_")[1]
    if spname not in species2motifs: 
        print(spname, "is missing")
        continue
    _df = df[(df[spcol]>=min_freq)&(df["chr"].isin(species2refs[spname]))]
    data = []
    # depth 6mA	5mC	mean 6mA freq	mean m5C freq	positions
    data.append(2*_df[spcol.replace("mod_frequency", "depth")].mean())
    # get number of every modification
    for mod in mod2mers.keys():
        data.append((_df["mod"]==mod).sum())
    # get avg mod_freq
    for mod in mod2mers.keys():
        data.append(_df.loc[_df["mod"]==mod, spcol].mean())
    # store total number of modifications
    data.append(len(_df))    
    # get each motif penetrance
    for m, mpos in species2motifs[spname].items():
        common = mpos.intersection(_df["ref"].to_numpy())
        data.append(len(common)/len(mpos))
    print(spname, *data)

ecoli 914.3166202003216 39021 28770 0.6673466338638169 0.43915676051442476 67791 0.9992679355783309 0.9966672221296451 0.9988427839312283 0.0005853636030715729
ecoli 251.54980645354138 38847 28062 0.6775930959919685 0.4950607226854822 66909 0.999477096841665 0.9983336110648225 0.9991734170937345 0.0009313913487296457


In [18]:
# #/home/lpryszcz/cluster/dna_mods/ecoli/modPhred.2020/PRJNA477598/
fn = "/home/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/mod.gz" # PRJNA477598 PRJEB22772
df = pd.read_csv(fn, sep="\t", skiprows=16)#; df.head() 
# get ref as chr:pos + strand
df["ref"] = df['chr'].astype(str) + ":" + df['pos'].astype(str) + df['strand']; df.head()

,chr,pos,ref_base,strand,mod,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/barcode01_cneoformans_pass.bam depth,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/barcode01_cneoformans_pass.bam basecall_accuracy,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/barcode01_cneoformans_pass.bam mod_frequency,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/barcode01_cneoformans_pass.bam median_mod_prob,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/barcode01_paeruginosa_pass.bam depth,...,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/barcode08_scerevisiae_pass.bam median_mod_prob,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/nasa_chiu_r9.bam depth,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/nasa_chiu_r9.bam basecall_accuracy,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/nasa_chiu_r9.bam mod_frequency,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/nasa_chiu_r9.bam median_mod_prob,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/nasa_mason_r9.bam depth,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/nasa_mason_r9.bam basecall_accuracy,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/nasa_mason_r9.bam mod_frequency,/users/enovoa/lpryszcz/cluster/dna_mods/ecoli/qsub/modPhred/PRJNA477598/minimap2/nasa_mason_r9.bam median_mod_prob,ref
0,NC_000913.3,73,C,+,5mC,0,NaN,NaN,NaN,1,...,NaN,6,1.000,0.167,0.600,15,1.0,0.000,NaN,NC_000913.3:73+
1,NC_000913.3,78,C,+,5mC,0,NaN,NaN,NaN,1,...,NaN,7,1.000,0.000,NaN,15,1.0,0.067,0.733,NC_000913.3:78+
2,NC_000913.3,235,A,+,5mC,0,NaN,NaN,NaN,1,...,NaN,8,1.000,0.000,NaN,15,0.8,0.067,0.900,NC_000913.3:235+
3,NC_000913.3,244,C,-,5mC,0,NaN,NaN,NaN,0,...,NaN,8,1.000,0.125,0.533,17,1.0,0.000,NaN,NC_000913.3:244-
4,NC_000913.3,290,C,-,5mC,0,NaN,NaN,NaN,0,...,NaN,8,0.875,0.125,0.867,19,1.0,0.000,NaN,NC_000913.3:290-


In [21]:
for spcol in filter(lambda c: c.endswith("mod_frequency"), df.columns):
    spname = os.path.basename(spcol.split()[0]).split("_")[1]
    if spname not in species2motifs: 
        print(spname, "is missing")
        continue
    _df = df[(df[spcol]>=min_freq)&(df["chr"].isin(species2refs[spname]))]
    data = []
    # depth 6mA	5mC	mean 6mA freq	mean m5C freq	positions
    data.append(2*_df[spcol.replace("mod_frequency", "depth")].mean())
    # store total number of modifications
    data.append(len(_df))    
    # get number of every modification
    for mod in mod2mers.keys():
        data.append((_df["mod"]==mod).sum())
    # get avg mod_freq
    for mod in mod2mers.keys():
        data.append(_df.loc[_df["mod"]==mod, spcol].mean())    
    # get each motif penetrance
    for m, mpos in species2motifs[spname].items():
        common = mpos.intersection(_df["ref"].to_numpy())
        data.append(len(common)/len(mpos))
    print(spname, *data)

cneoformans 66.69486946322435 75879 33262 42617 0.15330340929589323 0.11298118121876247 0.19445603493838493 0.0011776931070315206 0.0014449872703502374 0.02118646867957944
paeruginosa 148.2054072348125 76823 69723 7100 0.28282664257131795 0.07503281690140846 0.8204258848208406 0.00047188645685868817 0.0005306648472443333 0.0014650344243777535
ecoli 349.28877466172804 66145 37306 28839 0.6896688468342894 0.39400783661014604 0.9406504915289688 0.9395100816530578 0.9389981815176062 0.002652879383378557
ecoli 46.23712826218895 79088 39899 39189 0.6432901075214917 0.2992074306565618 0.9400230077389667 0.930178303616064 0.9358571664737974 0.007350206030686894
lfermentum 318.87232374404437 16581 15199 1382 0.22294078557799854 0.0747590448625181 0.7246293086847679 0.0 0.0 0.002357396237146578
efaecalis 378.84519606021183 5381 3858 1523 0.14688439606013481 0.07735521996060409 0.2609386281588448 0.0 0.0 0.005211597736775678
lmonocytogenes 199.7266338721012 5692 3314 2378 0.14550905250452625 0.07

In [12]:
for c in filter(lambda c: c.endswith("median_mod_prob"), df.columns):
    for mod in mod2mers.keys():
        print(os.path.basename(c.split()[0]), mod, df.loc[df["mod"]==mod, c].mean())

MARC_ZFscreens_R9.4_1D-Ecoli-run_FAF05145.bam 6mA 0.8780236143837723
MARC_ZFscreens_R9.4_1D-Ecoli-run_FAF05145.bam 5mC 0.865344574507618
MARC_ZFscreens_R9.4_2D-Ecoli-run_FAF05711.bam 6mA 0.879114881637876
MARC_ZFscreens_R9.4_2D-Ecoli-run_FAF05711.bam 5mC 0.8731131579760765


In [13]:
2 < 3 < 4

True